In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pickle
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
def pg_hook():
    return psycopg2.connect(dbname="mydb", user="myuser", password="mypassword", host='localhost', port=5439)
conn = pg_hook()
conn.autocommit = True

In [6]:
q_pg = """
SELECT * FROM mlm.scores;
"""
df = pd.read_sql(q_pg, conn)
df.set_index('puid', drop=True, inplace=True)
distance_ths = 2000
df['geo_similarity'] = df['geo_distance'].where(~df['geo_distance'].isnull(),distance_ths).apply(lambda r:max( (distance_ths-r)/distance_ths , 0))

In [7]:
X= df[['firstnames_tfidf', 'surname_tfidf', 'locality_tfidf',
       'postalcodelong_tfidf', 'route_tfidf', 'title_tfidf',
       'firstnames_levenshtein', 'surname_levenshtein', 'route_levenshtein',
       'locality_levenshtein', 'postalcodelong_levenshtein',
       'firstnames_trg_metaphone', 'surname_trg_metaphone',
       'locality_trg_metaphone', 'route_trg_metaphone',
       'phone_exact', 'email_exact', 'ni_number_exact', 'geo_similarity']].fillna(0)

## Load model

In [8]:
with open('/Users/pogier/Documents/63-Elk/dockersetup/shared_volume/ml_models/gradient_boosting.pickle' ,'rb') as f:
    gb = pickle.load(f)

In [10]:
y_true = pd.read_sql(""" SELECT puid, y_true FROM mlm.ytrue;""", conn).set_index('puid')

In [15]:
ix_common = y_true.index.intersection(X.index)

In [17]:
gb.score( X.loc[ix_common], y_true.loc[ix_common])

0.9974293059125964

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import  recall_score, accuracy_score, precision_score

In [29]:
y_pred = gb.predict(X.loc[ix_common])
recall_score(y_true.loc[ix_common], y_pred)

0.9959016393442623

In [30]:
precision_score(y_true.loc[ix_common], y_pred)

0.9959016393442623

## Go to Predict

In [21]:
y_proba = pd.Series(data=gb.predict_proba(X)[:,1], index=X.index, name='y_proba')


In [22]:
fp_export = '/Users/pogier/Documents/63-Elk/dockersetup/shared_volume/staging/y_proba.csv'

In [24]:
pd.DataFrame(y_proba).to_csv(fp_export, sep=',', encoding='utf-8', index=True)